# CT5170: Principles of ML - Assignment 2
## Course code: 1MAO3
### Participants (name: id): [ Daniel Verdejo: 22240224, Tom Cronin: < id >]


### TODO: DELETE THIS - ITS SOME BASIC INFORMATION AS A REFERENCE 

## Definition of the perceptron
The perceptron is a linear-model binary classifier with a simple input—output relationship as depicted in Figure 2-3, which shows we're summing n number of inputs times their associated weights and then sending this "net input" to a step function with a defined threshold. Typically with perceptrons, this is a Heaviside step function with a threshold value of 0.5. This function will output a real-valued single binary value (0 or a 1), depending on the input.

![perceptron](assets/perceptronFig.png?raw=true)

We can model the decision boundary and the classification output in the Heaviside
step function equation, as follows:

![f of x](assets/fx.png)

To produce the net input to the activation function (here, the Heaviside step function) we take the dot product of the input and the connection weights. We see this
summation in the left half of Figure 2-3 as the input to the summation function.
Table 2-1 provides an explanation of how the summation function is performed as
well as notes about the parameters involved in the summation function.


![params](assets/params.png)

The output of the step function (activation function) is the output for the perceptron and gives us a classification of the input values. If the bias value is negative, it forces the learned weights sum to be a much greater value to get a 1 classification output. The bias term in this capacity moves the decision boundary around for the model. Input values do not affect the bias term, but the bias term is learned through the perceptron learning algorithm.

## The perceptron learning algorithm
The perceptron learning algorithm changes the weights in the perceptron model until all input records are all correctly classified. The algorithm will not terminate if the learning input is not linearly separable. A linearly separable dataset is one for which we can find the values of a hyperplane that will cleanly divide the two classes of the dataset.

The perceptron learning algorithm initializes the weight vector with small random values or 0.0s at the beginning of training. The perceptron learning algorithm takes each input record, as we can see in Figure 2-3, and computes the output classification to check against the actual classification label. To produce the classification, the columns (features) are matched up to weights where n is the number of dimensions in both our input and weights. The first input value is the bias input, which is always 1.0 because we don't affect the bias input. The first weight is our bias term in this diagram. The dot product of the input vector and the weight vector gives us the input to our activation function, as we've previously discussed.

If the classification is correct, no weight changes are made. If the classification is incorrect, the weights are adjusted accordingly. Weights are updated between individual training examples in an "online learning" fashion. This loop continues until all of the input examples are correctly classified. If the dataset is not linearly separable, the training algorithm will not terminate. Figure 2-4 demonstrates a dataset that is not linearly separable, the XOR logic function.

![xor](assets/xor.png)

In [161]:
import pandas as pd
import numpy as np

import doctest


In [162]:
# Tom Cronin
def read_data_return_dataframe(PathToFile):
    """
    Should return a dataframe of the test file
    >>> type(read_data_return_dataframe('testdata.txt'))
    <class 'pandas.core.frame.DataFrame'>

     Should have correct data in the dataframe.
     There should be 20 entrys in the dataframe excluding of the column names
    >>> read_data_return_dataframe('testdata.txt').size
    20
    """
    return pd.read_table(PathToFile) # reads txt file and converts it to a pandas dataframe

In [163]:
doctest.run_docstring_examples(read_data_return_dataframe, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    type(read_data_return_dataframe('testdata.txt'))
Expecting:
    <class 'pandas.core.frame.DataFrame'>
ok
Trying:
    read_data_return_dataframe('testdata.txt').size
Expecting:
    20
ok


In [164]:
# Daniel Verdejo - split into labels and attributes
def split_df_labels_attributes(df):
    """ Split the dataframe into two by labels and attributes

        Keyword arguments:
        df -- A pandas dataframe type containing labels and attributes
        label_col_name -- A string which contains the name of the label column. 

        Returns:
        tuple -- (label: pd.DataFrame, attributes: pd.DataFrame)

        Should output only the Label column
        >>> labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata.txt'))
        >>> labels
                  Label
                0  no   
                1  no

        Should not contain the Label column
        >>> labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata-alt.txt'))
        >>> 'label' in attrs
        False
        
        >>> attrs.columns
        Index(['year', 'temp', 'humidity', 'rainfall', 'drought_code', 'buildup_index',
          'day', 'month', 'wind_speed'],
        dtype='object')
      """
    return (df.iloc[:,0:1], df.iloc[:,1:])  # (for every row take columns upto index 1 exclusive, for every row take every column from 1 onwards inclusive)

In [165]:
doctest.run_docstring_examples(split_df_labels_attributes, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata.txt'))
Expecting nothing
ok
Trying:
    labels
Expecting:
              Label
            0  no   
            1  no
ok
Trying:
    labels, attrs = split_df_labels_attributes(read_data_return_dataframe('testdata-alt.txt'))
Expecting nothing
ok
Trying:
    'label' in attrs
Expecting:
    False
ok
Trying:
    attrs.columns
Expecting:
    Index(['year', 'temp', 'humidity', 'rainfall', 'drought_code', 'buildup_index',
      'day', 'month', 'wind_speed'],
    dtype='object')
ok


In [166]:
# Daniel Verdejo - train, test splilt dataframe
def split_df_to_train_test_dfs(df):
    """ Splits a single dataframe into 2 dataframes
    
    Arguments:
    df -- A pandas Dataframe to be split into 2

    Returns:
    tuple -- (df_train: pandas.Dataframe, df_test: pandas.Dataframe)
    
    Should split a single dataframe into 2 unique dataframes
    >>> df = read_data_return_dataframe('wildfires.txt')
    >>> train, test = split_df_to_train_test_dfs(df)
    >>> train.index.equals(tr.index)
    False

    Length of train and test dataframes should equal the length of the orignal
    >>> df = read_data_return_dataframe('wildfires.txt')
    >>> train, test = split_df_to_train_test_dfs(df)
    >>> len(train) + len(test) == len(df)
    True

    Should contain different values
    >>> df = read_data_return_dataframe('wildfires.txt')
    >>> train, test = split_df_to_train_test_dfs(df)
    >>> train.values != test.values
    True
    """
    train_frac = round(np.random.uniform(.6, .7), 2) # get a random float for our training fraction
    df_train = df.sample(frac = train_frac) # randomly sample a fraction of the dataframe between 60 & 70 % of its entirety
    return (df_train,  df.drop(df_train.index)) # return the training data and the test data

In [167]:
doctest.run_docstring_examples(split_df_to_train_test_dfs, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    df = read_data_return_dataframe('wildfires.txt')
Expecting nothing
ok
Trying:
    train, test = split_df_to_train_test_dfs(df)
Expecting nothing
ok
Trying:
    train.index.equals(tr.index)
Expecting:
    False
ok
Trying:
    df = read_data_return_dataframe('wildfires.txt')
Expecting nothing
ok
Trying:
    train, test = split_df_to_train_test_dfs(df)
Expecting nothing
ok
Trying:
    len(train) + len(test) == len(df)
Expecting:
    True
ok
Trying:
    df = read_data_return_dataframe('wildfires.txt')
Expecting nothing
ok
Trying:
    train, test = split_df_to_train_test_dfs(df)
Expecting nothing
ok
Trying:
    train.values != test.values
Expecting:
    True
ok


<doctest NoName[8]>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  train.values != test.values


In [281]:
# Daniel Verdejo - initial perceptron based on description above 
# TODO : create a fn to normalise the outputs! THEN change this test to use the real y!!! ¯\_(ツ)_/¯
# TODO : The predict is also fucked right now ¯\_(ツ)_/¯
# TODO : Figure out if fit_predict and eval are needed ¯\_(ツ)_/¯
class Perceptron:
    """ The perceptron class 

    should return am imstamce of the created object
    >>> Perceptron(learn_rate=0.1, n_iters=100)
    Perceptron()

    should return self (an instance of the object)
    >>> df = read_data_return_dataframe("./testdata-alt.txt")
    >>> y, X = split_df_labels_attributes(df)
    >>> Perceptron(learn_rate=0.1, n_iters=100).fit(X, np.array([0,0,1,0,1,1]))
    Perceptron()

    should return the prediction
    >>> df = read_data_return_dataframe("./testdata-alt.txt")
    >>> y, X = split_df_labels_attributes(df)
    >>> _, X_test = split_df_labels_attributes(read_data_return_dataframe('./testdata.txt'))
    >>> P = Perceptron(learn_rate=0.1, n_iters=100).fit(X, np.array([0,0,1,0,1,1]))
    >>> pred = P.predict(X_test)
    [0, 0] 
    """

    def __init__(self, learn_rate, n_iters):
        self.learn_rate = learn_rate
        self.n_iters = n_iters
        self.activation_fn = (lambda x: np.where(x >= 0, 1, 0))
        self.weights = None
        self.bias = None

    def __repr__(self):
        return f"{type(self).__name__}()"

    def fit(self, X, y=None):

        if(isinstance(X, pd.DataFrame)):
            X = X.to_numpy()
        
        self.weights = np.random.rand(X.shape[1])
        self.bias = 0

        for _ in range(self.n_iters):
            for i, x_i in enumerate(X):
                ln_out = np.dot(x_i, self.weights) + self.bias
                pred = self.activation_fn(ln_out)
                update = self.learn_rate * (y[i] - pred)

                self.weights = self.weights + (update * x_i)
                self.bias += update
                
        return self

    def predict(self, X):
        if(isinstance(X, pd.DataFrame)):
            X = X.to_numpy()

        ln_out = np.dot(X, self.weights) + self.bias
        pred = self.activation_fn(ln_out)
        print(ln_out)
        return pred


    # Not sure if these are needed yet
    def fit_predict(self):
        return self
    
    def evaluate(self):
        pass

# TODO: remove these when done testing! 
# df = read_data_return_dataframe("./testdata-alt.txt")
# df_train, df_test = split_df_to_train_test_dfs(df)
# y_train, X_train = split_df_labels_attributes(df_train)

# y_train = [1 if y == 'yes' else 0 for y in y_train['label']]

# y_test, X_test = split_df_labels_attributes(df_test)

# P = Perceptron(.05, 5).fit(X_train, y_train)
# P
# # # pred = P.predict(X_test)
# # pred


Perceptron()

In [282]:
doctest.run_docstring_examples(Perceptron, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    Perceptron(learn_rate=0.1, n_iters=100)
Expecting:
    Perceptron()
ok
Trying:
    df = read_data_return_dataframe("./testdata-alt.txt")
Expecting nothing
ok
Trying:
    y, X = split_df_labels_attributes(df)
Expecting nothing
ok
Trying:
    Perceptron(learn_rate=0.1, n_iters=100).fit(X, np.array([0,0,1,0,1,1]))
Expecting:
    Perceptron()
ok
Trying:
    df = read_data_return_dataframe("./testdata-alt.txt")
Expecting nothing
ok
Trying:
    y, X = split_df_labels_attributes(df)
Expecting nothing
ok
Trying:
    _, X_test = split_df_labels_attributes(read_data_return_dataframe('./testdata.txt'))
Expecting nothing
ok
Trying:
    P = Perceptron(learn_rate=0.1, n_iters=100).fit(X, np.array([0,0,1,0,1,1]))
Expecting nothing
ok
Trying:
    pred = P.predict(X_test)
Expecting:
    0
**********************************************************************
File "__main__", line ?, in NoName
Failed example:
    pred = P.predict(X_test)
Expected:
    0
Got:
    [9377

![entropy](assets/entropy.png)

In [245]:
# Entropy calculation - need to figure out whats really required to check what causes a fire or not
# Presuming the date isn't really all that important, I believe the attributes of interest are:
# temp, humidity, rainfall, drought_code, buildup_index, wind_speed
# For each outcome, (yes / no in this case) sum all of the positive and negative cases to the first algorithm above 
# page 139 - ref: https://drive.google.com/drive/folders/1U6F01iWnnSTrJQzviE02ST8WMc8UdOML

def entropy(y):
    """ Entropy - for a set of data summate all positive cases, and negative cases:
        for each case calculate p * log2 p (log base 2) if p is a non negative number

        Should return 1.0 if there label distribution is equal
        >>> round(entropy(np.array([1,1,1,1,0,0,0,0])),3)
        1.0

        Should return 0.0 if the labels are one sided negatively
        >>> Y = np.array([0,0,0,0,0,0,0,0,0])
        >>> round(entropy(Y),3)
        -0.0
        
        Should return 0.0 if the labels are one sided positively
        >>> Y = np.array([1,1,1,1,1])
        >>> round(entropy(Y),3)
        -0.0

        Should return a value close to 1.0 if the labels are almost equal
        >>> Y = np.array([1, 0, 0, 0, 1, 1, 0 ,1 ,0, 0, 1, 0, 1])
        >>> round(entropy(Y), 3)
        0.996

        Should return a float closer to 0.0 if the labels are very unbalanced
        >>> Y = np.array([1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1])
        >>> round(entropy(Y),3)
        0.286
    """
    P = np.bincount(y) / len(y) # number of unique outputs / all possible outputs
    return - np.sum([p * np.log2(p) for p in P if p > 0])


In [246]:
doctest.run_docstring_examples(entropy, globals(), verbose=True, optionflags=doctest.NORMALIZE_WHITESPACE)

Finding tests in NoName
Trying:
    round(entropy(np.array([1,1,1,1,0,0,0,0])),3)
Expecting:
    1.0
ok
Trying:
    Y = np.array([0,0,0,0,0,0,0,0,0])
Expecting nothing
ok
Trying:
    round(entropy(Y),3)
Expecting:
    -0.0
ok
Trying:
    Y = np.array([1,1,1,1,1])
Expecting nothing
ok
Trying:
    round(entropy(Y),3)
Expecting:
    -0.0
ok
Trying:
    Y = np.array([1, 0, 0, 0, 1, 1, 0 ,1 ,0, 0, 1, 0, 1])
Expecting nothing
ok
Trying:
    round(entropy(Y), 3)
Expecting:
    0.996
ok
Trying:
    Y = np.array([1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1])
Expecting nothing
ok
Trying:
    round(entropy(Y),3)
Expecting:
    0.286
ok
